In [1]:
import numpy as np
import time
from io import StringIO
import bqplot as bq
import ipywidgets as ipw
import ipyvolume as ipv
import IPython

In [3]:
def split_by_key(new, key, start_ind=0):
    "Split into before and after at first newline before `key`"
    # str.find returns -1 if not found
    ind = new[start_ind:].find(key)
    if ind != -1:
        # Find last \n before `key`
        reverse_str = new[ind-1::-1]
        reverse_ind = reverse_str.find('\n')
        # If \n isn't found, then all of `new` belongs
        # in the next group
        if reverse_ind == -1:
            newline_ind = 0
        else:
            newline_ind = ind - reverse_ind
            
        # Append text before `key` to `lines`
        before = new[:newline_ind]
        after = new[newline_ind:]
    else:
        before = new
        after = ''
        
    return before, after, ind


In [4]:
def parse_timestep(ts_string):
    lines = ts_string.split('\n')
    header = lines[:9]
    bounds_io = StringIO('\n'.join(lines[5:8]))
    data_io = StringIO('\n'.join(lines[9:]))
    bounds_arr = np.loadtxt(bounds_io) 
    try:
        data_arr = np.loadtxt(data_io)
    except ValueError:
        print("VE")
        IPython.embed()
    
    data_min, data_max = bounds_arr.T
    data_range = data_max - data_min
    
    atom_id, atom_type, xs, ys, zs, ix, iy, iz = data_arr.T
    sc = data_arr[:,[2,3,4]]
    im = data_arr[:,[5,6,7]]
    #pos = data_range * sc + data_min*im
    pos = sc
    
    return pos

In [5]:
def poll_file(path, func, key, poll_interval=1, ignore_first=True, *args, **kwargs):
    """
    Poll `path` every `poll_interval` seconds.
    Every time `key` is contained, call `func`,
    passing the full text read since the last time `key` was read
    as the argument. Then throw away what we've read before last `key`.
    
    args, kwargs will be passed to func.
    """
    
    lines = ''
    
    ignore = ignore_first
    
    while len(lines) == 0:
        try:
            with open(path) as fh:
                while True:
                    new = fh.read(256)
                    if len(new) > 0:
                        before, after, ind = split_by_key(new, key)
                        
                        # If `key` in `new`
                        if ind != -1:
                            if new.count(key) != 1:
                                print("count = {}".format(new.count(key)))
                                IPython.embed()
                            if ignore:
                                lines += new
                                ignore = False
                            else:
                                lines += before
                                func(lines, *args, **kwargs)
                                lines = after
                                    
                        else:
                            lines += new
                    else:
                        time.sleep(poll_interval)
                        
        except FileNotFoundError:
            print("No file yet.")
            time.sleep(1)


In [6]:
def g_of_r(atom_pos, dr, r_max):
    """
    atom_pos :: N x 3 array of atom positions
    dr :: bin thickness
    """
    
    n = atom_pos.shape[0]
    num_bins = int(np.floor(r_max / dr))
    r_vals = np.arange(0, r_max, dr)
    diff = np.zeros(n*(n-1))

    i, j = np.meshgrid(range(n), range(n))
    i = i.reshape(n**2)
    j = j.reshape(n**2)
    
    diff = np.linalg.norm(atom_pos[i,:] - atom_pos[j,:], axis=1)
    diff = diff[np.nonzero(diff)]
    
    g_vals, _ = np.histogram(diff, bins=num_bins, range=(0, r_max), density=True)
    
    return r_vals, g_vals

In [7]:
def create_bq_figure(x, y):
    xsc = bq.LinearScale()
    ysc = bq.LinearScale()
    xax = bq.Axis(scale=xsc, label='r')
    yax = bq.Axis(scale=ysc, orientation='vertical', label='g(r)')
    
    line = bq.Lines(
        x=x,
        y=y,
        scales={
            'x': xsc,
            'y': ysc
        }
    )
    
    fig = bq.Figure(
        axes = [xax, yax],
        marks = [line],
        #animation_duration=500
    )
    
    return line, fig

In [8]:
def create_ipv_scatter(xlim, ylim, zlim):
    fig = ipv.figure()
    scatter = ipv.scatter(*np.zeros([3,1]))
    
    fig.xlim = xlim
    fig.ylim = ylim
    fig.zlim = zlim
    
    fig.animation = 0
    
    return scatter, fig

In [9]:
def update_line(line, y):
    line.y = y

In [10]:
def update_ipv_scatter(scatter, atoms):
    scatter.x = atoms[:,0]
    scatter.y = atoms[:,1]
    scatter.z = atoms[:,2]

In [11]:
def update_atom_plots(atoms_str, line, scatter):
    atoms = parse_timestep(atoms_str)
    r, g = g_of_r(atoms, dr=.01, r_max=2)
    update_line(line, g)
    update_ipv_scatter(scatter, atoms)

In [12]:
r_max = 2
dr = .01
r = np.arange(0, r_max, dr)
g = np.zeros_like(r)

In [13]:
line, bq_fig = create_bq_figure(r, g)
scatter, ipv_fig = create_ipv_scatter(
    (0,1),
    (0,1),
    (0,1)
)

ipw.HBox([bq_fig, ipv_fig])

HBox(children=(Figure(axes=[Axis(label='r', scale=LinearScale()), Axis(label='g(r)', orientation='vertical', scale=LinearScale())], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(min_width='125px'), marks=[Lines(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([ 0.  ,  0.01,  0.02,  0.03,  0.04,  0.05,  0.06,  0.07,  0.08,
        0.09,  0.1 ,  0.11,  0.12,  0.13,  0.14,  0.15,  0.16,  0.17,
        0.18,  0.19,  0.2 ,  0.21,  0.22,  0.23,  0.24,  0.25,  0.26,
        0.27,  0.28,  0.29,  0.3 ,  0.31,  0.32,  0.33,  0.34,  0.35,
        0.36,  0.37,  0.38,  0.39,  0.4 ,  0.41,  0.42,  0.43,  0.44,
        0.45,  0.46,  0.47,  0.48,  0.49,  0.5 ,  0.51,  0.52,  0.53,
        0.54,  0.55,  0.56,  0.57,  0.58,  0.59,  0.6 ,  0.61,  0.62,
        0.63,  0.64,  0.65,  0.66,  0.67,  0.68,  0.69,  0.7 ,  0.71,
        0.72,  0.73,  0.74,  0.75,  0.76,  0.77,  0.78,  0.79,  0.8 ,
        0.81,  0.82,  0.83,  0.84,  0.85,  0.86,  0.87,  0.88,  0.89,
        0.9 ,  0.91,  0.92,  0.93,  0.94,  0.95,  0.96,  0.97,  0.98,
        0.99,  1.  ,  1.01,  1.02,  1.03,  1.04,  1.05,  1.06,  1.07,
        1.08,  1.09,  1.1 ,  1.11,  1.12,  1.13,  1.14,  1.15,  1.16,
        1.17,  1.18,  1.19,  1.2 ,  1.21,  1.22,  1.23,  1.24,  1.25,
        1.26,  1.27,  1.28,  1.29,  1.3 ,  1.31,  1.32,  1.33,  1.34,
        1.35,  1.36,  1.37,  1.38,  1.39,  1.4 ,  1.41,  1.42,  1.43,
        1.44,  1.45,  1.46,  1.47,  1.48,  1.49,  1.5 ,  1.51,  1.52,
        1.53,  1.54,  1.55,  1.56,  1.57,  1.58,  1.59,  1.6 ,  1.61,
        1.62,  1.63,  1.64,  1.65,  1.66,  1.67,  1.68,  1.69,  1.7 ,
        1.71,  1.72,  1.73,  1.74,  1.75,  1.76,  1.77,  1.78,  1.79,
        1.8 ,  1.81,  1.82,  1.83,  1.84,  1.85,  1.86,  1.87,  1.88,
        1.89,  1.9 ,  1.91,  1.92,  1.93,  1.94,  1.95,  1.96,  1.97,
        1.98,  1.99]), y=array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.]))], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0)), Figure(animation=0.0, camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color_selected=array('white',
      dtype='<U5'), size=array(2), size_selected=array(2.6), x=array([ 0.]), y=array([ 0.]), z=array([ 0.]))], style={'axes': {'visible': True, 'label': {'color': 'black'}, 'ticklabel': {'color': 'black'}, 'color': 'black'}, 'box': {'visible': True}, 'ba

In [14]:
poll_file('lammps_melt/melt.lammpstrj', update_atom_plots, 'TIMESTEP', line=line, scatter=scatter)

No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.
No file yet.

KeyboardInterrupt: 

In [ ]:
# def poll_thermo(path, func, key):
#     lines = ''
#     
#     ignore = ignore_first
#     
#     while len(lines) == 0:
#         try:
#             with open(path) as fh:
#                 while True:
#                     new = fh.read(256)
#                     if len(new) > 0:
#                         before, after, ind = split_by_key(new, key)
#                         
#                         # If `key` in `new`
#                         if ind != -1:
#                             if ignore:
#                                 lines += new
#                                 ignore = False
#                             else:
#                                 lines += before
#                                 func(lines, *args, **kwargs)
#                                 lines = after
#                         else:
#                             lines += new
#                     else:
#                         time.sleep(poll_interval)
#                         
#         except FileNotFoundError:
#             print("No file yet.")
#             time.sleep(1)